In [43]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import requests

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [2]:
tf.__version__

'2.9.0'

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Metal device set to: Apple M1 Pro


2023-02-22 12:54:09.843550: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-22 12:54:09.843867: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


True

# get data

In [5]:
def get_ds():
    url = "https://datasets-server.huggingface.co/parquet?dataset=aadityaubhat%2FGPT-wiki-intro"
    response = requests.get(url)
    if response.status_code !=200:
        return f"error during dataset request: {response.status_code}"
    
    url_parquet = [files['url'] for files in response.json()['parquet_files']]
    
    
    df = [pd.read_parquet(url_) for url_ in url_parquet]
    
    return pd.concat(df)

In [6]:
df = get_ds()
df.set_index('id',inplace=True)

In [7]:
df['random']=np.random.random(len(df))

In [8]:
# reorganize ds and randomize samples wiki/generated
df.loc[df['random']<.5,'text']=df['generated_intro']
df.loc[df['random']<.5,'label']='generated'
df.loc[df['random']>=.5,'text']=df['wiki_intro']
df.loc[df['random']>=.5,'label']='wiki'

In [9]:
df.label.value_counts()

wiki         75039
generated    74961
Name: label, dtype: int64

In [10]:
#check distribution is similar accross classes

In [11]:
df[df['label']=='generated'].describe()

,title_len,wiki_intro_len,generated_intro_len,prompt_tokens,generated_text_tokens,random
count,74961.000000,74961.000000,74961.000000,74961.000000,74961.000000,74961.000000
mean,2.215472,196.005389,129.293739,28.956297,165.494964,0.250315
std,0.630953,41.422459,56.985389,5.026033,77.129319,0.144519
min,1.000000,150.000000,7.000000,21.000000,1.000000,0.000010
25%,2.000000,164.000000,86.000000,26.000000,106.000000,0.125203
50%,2.000000,184.000000,122.000000,28.000000,154.000000,0.250413
75%,3.000000,216.000000,170.000000,31.000000,220.000000,0.375184
max,3.000000,350.000000,278.000000,148.000000,300.000000,0.499988


In [12]:
df[df['label']=='wiki'].describe()

,title_len,wiki_intro_len,generated_intro_len,prompt_tokens,generated_text_tokens,random
count,75039.000000,75039.000000,75039.000000,75039.000000,75039.000000,75039.000000
mean,2.215288,196.004837,129.644372,28.959221,165.986287,0.749148
std,0.628077,41.466319,57.095142,5.063986,77.312752,0.144812
min,1.000000,150.000000,7.000000,21.000000,2.000000,0.500002
25%,2.000000,164.000000,86.000000,26.000000,107.000000,0.623386
50%,2.000000,184.000000,122.000000,28.000000,155.000000,0.748501
75%,3.000000,216.000000,171.000000,31.000000,221.000000,0.874417
max,3.000000,350.000000,274.000000,141.000000,300.000000,0.999998


In [13]:
X_train,X_test,y_train,y_test = train_test_split(df['text'],df['label'],test_size=.2)

In [14]:
y_train

id
22367823         wiki
67428769    generated
68402160    generated
2642042     generated
4341348     generated
              ...    
58444157    generated
12569126         wiki
16694673    generated
1291347     generated
5080244          wiki
Name: label, Length: 120000, dtype: object

# Baseline

In [15]:
# vectorize text ; using tfidf
vectorize = TfidfVectorizer(ngram_range=(1,1),max_df=.95,min_df=.05)

#transform
X_train_vect = vectorize.fit_transform(X_train)
X_test_vect = vectorize.transform(X_test)

In [16]:
X_train_vect.shape

(120000, 213)

In [17]:
#NB
model = MultinomialNB()
model.fit(X_train_vect,y_train)

MultinomialNB()

In [18]:
# accuracy of NB model
model.score(X_test_vect,y_test)

0.7497

In [19]:
# classification report 
y_pred = model.predict(X_test_vect)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

   generated       0.73      0.77      0.75     14354
        wiki       0.77      0.73      0.75     15646

    accuracy                           0.75     30000
   macro avg       0.75      0.75      0.75     30000
weighted avg       0.75      0.75      0.75     30000



# Features engineering

In [20]:
#WIP

# Models

## Simple LSTM

In [21]:
try:
    X_train_vect = X_train_vect.toarray()
    X_test_vect = X_test_vect.toarray()
except AttributeError:
    pass

if len(X_train_vect.shape)<3:
    X_train_vect = np.expand_dims(X_train_vect,-1)
    X_test_vect = np.expand_dims(X_test_vect,-1)

y_train = np.where(y_train=='generated',1,0)
y_test = np.where(y_test=='generated',1,0)

In [42]:
def lstm_model():
    inputs = tf.keras.layers.Input(shape=(X_train_vect[0].shape))
#     inputs = tf.keras.layers.Input(shape=(200,1))
#     x = tf.keras.layers.LSTM(128,return_sequences=True,activation='tanh')(inputs)
    x = tf.keras.layers.LSTM(32,return_sequences=False,activation='tanh')(inputs)
    x = tf.keras.layers.Dense(16,activation='relu')(x)
    outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)
    
    return tf.keras.Model(inputs=inputs,outputs=outputs)

In [55]:
#compile
lstm = lstm_model()
lstm.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

In [62]:
#callbacks

#early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=4,restore_best_weights=True)

#tensorboard
log_dir = os.path.join(os.path.abspath(os.path.pardir),'logs','simple_lstm')
tb = tf.keras.callbacks.TensorBoard(log_dir=log_dir,write_graph=False,)

#learning rate
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=.5,patience=3,verbose=1)

In [ ]:
history = lstm.fit(x=X_train_vect,
                   y=np.expand_dims(y_train,-1),
                   validation_split=.2,
                   callbacks=[es,tb,lr],
                   batch_size=32,
                   epochs=50,)

Epoch 1/50
3000/3000 [==============================] - 167s 56ms/step - loss: 0.5606 - accuracy: 0.7086 - val_loss: 0.5541 - val_accuracy: 0.7103 - lr: 5.0000e-04
Epoch 2/50
3000/3000 [==============================] - 168s 56ms/step - loss: 0.5544 - accuracy: 0.7125 - val_loss: 0.5461 - val_accuracy: 0.7162 - lr: 5.0000e-04
Epoch 3/50
3000/3000 [==============================] - 168s 56ms/step - loss: 0.5470 - accuracy: 0.7179 - val_loss: 0.5421 - val_accuracy: 0.7186 - lr: 5.0000e-04
Epoch 4/50
3000/3000 [==============================] - 168s 56ms/step - loss: 0.5436 - accuracy: 0.7224 - val_loss: 0.5391 - val_accuracy: 0.7205 - lr: 5.0000e-04
Epoch 5/50
3000/3000 [==============================] - 169s 56ms/step - loss: 0.5373 - accuracy: 0.7265 - val_loss: 0.5257 - val_accuracy: 0.7348 - lr: 5.0000e-04
Epoch 6/50
3000/3000 [==============================] - 171s 57ms/step - loss: 0.5345 - accuracy: 0.7293 - val_loss: 0.5228 - val_accuracy: 0.7350 - lr: 5.0000e-04
Epoch 7/50
3000/

In [64]:
lstm.save(os.path.join(os.path.abspath(os.path.pardir),'models','simple_lstm'))

INFO:tensorflow:Assets written to: /Users/arthurcollard/code/arthurcol/gpt_vs_human/models/simple_lstm/assets


INFO:tensorflow:Assets written to: /Users/arthurcollard/code/arthurcol/gpt_vs_human/models/simple_lstm/assets


## Using Bidirectional LSTM

In [65]:
def bilstm_model():
    inputs = tf.keras.layers.Input(shape=(X_train_vect[0].shape))
#     inputs = tf.keras.layers.Input(shape=(200,1))
#     x = tf.keras.layers.LSTM(128,return_sequences=True,activation='tanh')(inputs)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,return_sequences=False,activation='tanh'))(inputs)
    x = tf.keras.layers.Dense(16,activation='relu')(x)
    outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)
    
    return tf.keras.Model(inputs=inputs,outputs=outputs)

In [69]:
#compile
bilstm = bilstm_model()
bilstm.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

In [70]:
#callbacks

#early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=4,restore_best_weights=True)

#tensorboard
log_dir = os.path.join(os.path.abspath(os.path.pardir),'logs','bi_lstm')
tb = tf.keras.callbacks.TensorBoard(log_dir=log_dir,write_graph=False,)

#learning rate
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=.5,patience=3,verbose=1)

In [ ]:
history = bilstm.fit(x=X_train_vect,
                   y=np.expand_dims(y_train,-1),
                   validation_split=.2,
                   callbacks=[es,tb,lr],
                   batch_size=32,
                   epochs=50,)

Epoch 1/50


2023-02-22 15:13:06.858445: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-22 15:13:07.012362: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-22 15:13:07.012416: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-22 15:13:07.604241: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-22 15:13:07.618011: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3000/3000 [==============================] - ETA: 0s - loss: 0.6707 - accuracy: 0.5787

2023-02-22 15:17:53.699522: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-22 15:17:53.764278: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-22 15:17:53.764332: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3000/3000 [==============================] - 331s 110ms/step - loss: 0.6707 - accuracy: 0.5787 - val_loss: 0.6152 - val_accuracy: 0.6658 - lr: 0.0010
Epoch 2/50
3000/3000 [==============================] - 326s 109ms/step - loss: 0.6138 - accuracy: 0.6689 - val_loss: 0.5840 - val_accuracy: 0.6930 - lr: 0.0010
Epoch 3/50
3000/3000 [==============================] - 331s 110ms/step - loss: 0.5833 - accuracy: 0.6964 - val_loss: 0.5585 - val_accuracy: 0.7130 - lr: 0.0010
Epoch 4/50
3000/3000 [==============================] - 334s 111ms/step - loss: 0.5705 - accuracy: 0.7060 - val_loss: 0.5519 - val_accuracy: 0.7225 - lr: 0.0010
Epoch 5/50
3000/3000 [==============================] - 329s 110ms/step - loss: 0.5597 - accuracy: 0.7134 - val_loss: 0.5473 - val_accuracy: 0.7209 - lr: 0.0010
Epoch 6/50
3000/3000 [==============================] - 332s 111ms/step - loss: 0.5495 - accuracy: 0.7224 - val_loss: 0.5292 - val_accuracy: 0.7365 - lr: 0.0010
Epoch 7/50
3000/3000 [=======================

## Unleash the power of attention models